In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from re import M

pd.set_option("display.max_columns", None)

In [8]:
# Get approximate number of available training examples

swan = pd.read_csv('data/SWAN.csv')
whodas = pd.read_csv('data/WHODAS.csv')
diagnoses = pd.read_csv('data/diagnoses-data.csv')
diagnoses = diagnoses.rename(columns = lambda column: column.split(",")[-1])
# Ignore the extra extra header
swan = swan.iloc[1:, :]
whodas = whodas.iloc[1:, :]

In [9]:
merged_swan_diag = pd.merge(swan, diagnoses, on="EID")
merged_whodas_diag = pd.merge(whodas, diagnoses, on="EID")

In [10]:
merged_swan_diag["Anonymized ID"].count()

3147

In [11]:
merged_swan_diag["Anonymized ID"].nunique()

3147

In [12]:
merged_whodas_diag["Anonymized ID"].count()

2531

In [13]:
merged_whodas_diag["Anonymized ID"].nunique()

2531

In [16]:
# How much data before November 2016 (WHODAS added)
merged_swan_diag[(merged_swan_diag["Year_x"]=="2016") & ((merged_swan_diag["Season_x"]=="Winter") | (merged_swan_diag["Season_x"] == "Spring") | (merged_swan_diag["Season_x"] == "Summer") )]["Anonymized ID"].count()

328

In [17]:
# Start from winter 2016 

merged_swan_diag_dropped = merged_swan_diag.drop(merged_swan_diag[
    (merged_swan_diag["Year_x"] == "2016") & (
        (merged_swan_diag["Season_x"] == "Winter") | 
        (merged_swan_diag["Season_x"] == "Spring") | 
        (merged_swan_diag["Season_x"] == "Summer") ) ].index)

# Remove missing diagnoses
merged_swan_diag_dropped = merged_swan_diag_dropped.drop(merged_swan_diag_dropped[
    (merged_swan_diag_dropped["DX_01"] == "No Diagnosis Given: Incomplete Eval")].index)

merged_swan_diag_dropped.count()[0]

2732

In [18]:
# Diagnosis counts (any position)
diag_colunms = ["DX_01", "DX_02", "DX_03", "DX_04", "DX_05", "DX_06", "DX_07", "DX_08", "DX_09", "DX_10"]
pd.set_option('display.max_rows', None) 
merged_swan_diag_dropped[diag_colunms].apply(pd.Series.value_counts).sum(numeric_only=True, axis=1).sort_values(ascending=False)

ADHD-Combined Type                                                             781.0
ADHD-Inattentive Type                                                          764.0
Specific Learning Disorder with Impairment in Reading                          504.0
Autism Spectrum Disorder                                                       476.0
Generalized Anxiety Disorder                                                   411.0
Oppositional Defiant Disorder                                                  392.0
Language Disorder                                                              358.0
Specific Phobia                                                                322.0
Social Anxiety (Social Phobia)                                                 285.0
Specific Learning Disorder with Impairment in Mathematics                      236.0
No Diagnosis Given                                                             233.0
Enuresis                                                         

In [24]:
# Diagnosis combinations counts
def clean_diag_list(row):
    row_without_duplicates = list({x for x in set(row) if x==x})
    row_without_duplicates.sort()
    return str(row_without_duplicates)

list_of_diagnosis_sets = list(map(clean_diag_list, merged_swan_diag_dropped[diag_colunms].to_numpy()))

counter = Counter(list_of_diagnosis_sets).most_common()
diag_combination_counter_df = pd.DataFrame.from_dict(counter)

pd.set_option('display.max_colwidth', None)

diag_combination_counter_df.head(10000)

,0,1
0,['No Diagnosis Given'],233
1,['ADHD-Inattentive Type'],153
2,['ADHD-Combined Type'],122
3,['Specific Learning Disorder with Impairment in Reading'],60
4,"['ADHD-Combined Type', 'Oppositional Defiant Disorder']",55
5,"['ADHD-Inattentive Type', 'Autism Spectrum Disorder']",41
6,"['ADHD-Combined Type', 'Autism Spectrum Disorder']",39
7,['Other Specified Attention-Deficit/Hyperactivity Disorder'],38
8,['Autism Spectrum Disorder'],31
9,"['ADHD-Inattentive Type', 'Specific Learning Disorder with Impairment in Reading']",26


In [ ]:
pd.set_option('display.max_rows', 20) 

## Explore questionnaire distribution
### LORIS data

In [110]:
# LORIS saved query (all data)
full = pd.read_csv("data/all.csv", dtype=object)

In [111]:
# Check which questionnaires are in the rows with unserscores (NDARZZ007YMP_1, NDARAA075AMK_Visit_1)
rows_with_underscore_in_id = full[full["Identifiers"].str.contains("_")]
non_empty_columns = rows_with_underscore_in_id.columns[
    ~rows_with_underscore_in_id.isin(["."]).all()].tolist()
non_empty_questionnaires_in_underscore = set([x.split(",")[0] for x in non_empty_columns])
non_empty_questionnaires_in_underscore

{'DailyMeds', 'Identifiers', 'TRF', 'TRF_P', 'TRF_Pre'}

In [112]:
# Drop rows with underscores and the questionnaires present in rows with underscores 
non_empty_questionnaires_in_underscore.remove("Identifiers")
main = full[~full["Identifiers"].str.contains("_")]
len(main.columns)

7120

In [113]:
filtered_main = main
for questionnaire in non_empty_questionnaires_in_underscore:
    filtered_main = filtered_main.drop(filtered_main.filter(regex=(questionnaire+",")), axis=1)
    len(filtered_main.columns)

In [114]:
# Exract ID
filtered_main["ID"] = filtered_main["Identifiers"].str.split(',').str[0].str.split('_').str[0]

# Drop IDs with quotes around them (duplicates)
filtered_main[filtered_main["ID"].str.contains("'")]
filtered_main = filtered_main.drop(filtered_main[filtered_main["ID"].str.contains("'")].index)

In [115]:
filtered_main["Identifiers"].count()

3630

In [116]:
filtered_main["ID"].count()

3630

In [117]:
filtered_main["ID"].nunique()

3630

In [118]:
# Replace . with NaN
filtered_main = filtered_main.replace(".", np.nan)

In [119]:
# How many ppl answered all questionnaires (Data_entry value for every questionnaire is not empty)
data_entry_columns = filtered_main.filter(regex=(",Data_entry"))
data_entry_columns.dropna(how = 'any')

,"ACE,Data_entry","APQ_P,Data_entry","APQ_SR,Data_entry","ARI_P,Data_entry","ARI_S,Data_entry","ASR,Data_entry","ASSQ,Data_entry","Audit,Data_entry","BIA,Data_entry","Barratt,Data_entry","Basic_Demos,Data_entry","C3SR,Data_entry","CAARS,Data_entry","CBCL,Data_entry","CBCL_Pre,Data_entry","CCSC,Data_entry","CDI_P,Data_entry","CDI_SR,Data_entry","CELF,Data_entry","CELF5_Meta,Data_entry","CELF_Full_5to8,Data_entry","CELF_Full_9to21,Data_entry","CELF_Meta,Data_entry","CGAS,Data_entry","CIS_P,Data_entry","CIS_SR,Data_entry","CPIC,Data_entry","CTOPP,Data_entry","ColorVision,Data_entry","ConsensusDx,Data_entry","DSM5_Dx,Data_entry","DTS,Data_entry","DigitSpan,Data_entry","DrugScreen,Data_entry","EEG_Track,Data_entry","EHQ,Data_entry","ESPAD,Data_entry","ESWAN,Data_entry","EVT,Data_entry","FFQ,Data_entry","FGC,Data_entry","FSQ,Data_entry","FTND,Data_entry","FTQA,Data_entry","GFTA,Data_entry","IAT,Data_entry","ICU_P,Data_entry","KBIT,Data_entry","MFQ_P,Data_entry","MFQ_SR,Data_entry","MRI_Track,Data_entry","NIDA,Data_entry","NIH5,Data_entry","NIH7,Data_entry","NLES_P,Data_entry","NLES_SR,Data_entry","PAQ_A,Data_entry","PAQ_C,Data_entry","PBQ,Data_entry","PCIAT,Data_entry","PPS,Data_entry","PPVT,Data_entry","PSI,Data_entry","Pegboard,Data_entry","PhenX_Neighborhood,Data_entry","PhenX_SchoolRisk,Data_entry","Physical,Data_entry","PreInt_Demos_Fam,Data_entry","PreInt_Demos_Home,Data_entry","PreInt_DevHx,Data_entry","PreInt_EduHx,Data_entry","PreInt_FamHx,Data_entry","PreInt_FamHx_RDC,Data_entry","PreInt_Lang,Data_entry","PreInt_TxHx,Data_entry","Pregnancy,Data_entry","Quotient,Data_entry","Quotient_Ext,Data_entry","RANRAS,Data_entry","RBS,Data_entry","SAS,Data_entry","SCARED_P,Data_entry","SCARED_SR,Data_entry","SCQ,Data_entry","SDQ,Data_entry","SDS,Data_entry","SRS,Data_entry","SRS_Pre,Data_entry","STAI,Data_entry","SWAN,Data_entry","SympChck,Data_entry","TOWRE,Data_entry","Tanner,Data_entry","Vineland,Data_entry","WAIS,Data_entry","WASI,Data_entry","WHODAS_P,Data_entry","WHODAS_SR,Data_entry","WIAT,Data_entry","WISC,Data_entry","YFAS,Data_entry","YFAS_C,Data_entry","YSR,Data_entry"


In [120]:
# Get questionnaires most people answered
pd.set_option('display.max_rows', None)
questionnaire_answer_counts = data_entry_columns.count().sort_values(ascending=False).to_frame()
questionnaire_answer_counts["Ratio"] = questionnaire_answer_counts[0]/filtered_main["ID"].nunique()*100
questionnaire_answer_counts

,0,Ratio
"Basic_Demos,Data_entry",3630,100.000000
"PreInt_EduHx,Data_entry",3559,98.044077
"PreInt_DevHx,Data_entry",3553,97.878788
"WIAT,Data_entry",3549,97.768595
"ColorVision,Data_entry",3524,97.079890
"Pegboard,Data_entry",3515,96.831956
"Physical,Data_entry",3503,96.501377
"FGC,Data_entry",3483,95.950413
"PreInt_TxHx,Data_entry",3477,95.785124
"CELF,Data_entry",3463,95.399449


In [121]:
# List of top mental health quetsionnaires and output variables (consensus diagnosis and impairment)
top_mh_assessments = ['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL', 'NLES_P', 
'SCARED_P', 'ICU_P', 'PCIAT', 'DTS', 'ESWAN', 'MFQ_P', 'YFAS_C', 'WHODAS_P', 'SDS', 'CIS_P', 'SAS', 'WHODAS_SR', 
'CIS_SR', 'CGAS', 'RBS', 'C3SR', 'SCARED_SR', 'MFQ_SR', 'ConsensusDx', 'CCSC', 'YSR', 'CDI_P', 'CDI_SR', 'YFAS', 
'CBCL_Pre', 'SRS_Pre', 'ASR', 'CAARS', 'STAI']

In [127]:
# get number of people who took all top 1, top 2, top 3, etc. popular assessments
cumul_number_of_examples_list = []
for i in range(1, len(top_mh_assessments)+1):
    columns = [x+",Data_entry" for x in top_mh_assessments[0:i]]
    cumul_number_of_examples = data_entry_columns[columns].notnull().all(axis=1).sum()
    print([x.split(",")[0] for x in columns], ": ", cumul_number_of_examples)
    cumul_number_of_examples_list.append(cumul_number_of_examples)

['ARI_S'] :  3276
['ARI_S', 'SympChck'] :  3069
['ARI_S', 'SympChck', 'SCQ'] :  3055
['ARI_S', 'SympChck', 'SCQ', 'ASSQ'] :  3033
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ'] :  2981
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P'] :  2964
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN'] :  2890
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS'] :  2813
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL'] :  2812
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL', 'NLES_P'] :  2638
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL', 'NLES_P', 'SCARED_P'] :  2495
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL', 'NLES_P', 'SCARED_P', 'ICU_P'] :  2416
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL', 'NLES_P', 'SCARED_P', 'ICU_P', 'PCIAT'] :  2404
['ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL', 'NLES_P', 'SC

Only 420 people who completed most popular questionnaires up to ConsensusDx

In [17]:
# Check if there are more complete diagnoses in COINS assessment data
diag = pd.read_csv('data/assessment_data/9994_ConsensusDx_20211130.csv')
diag.count()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3,7,8,10,16,17,18,19,20,21,22,23,24,25,31,32,33,34,35,36,37,38,39,40,46,47,48,49,50,51,52,53,54,55,61,62,63,64,65,66,67,68,69,70,76,77,78,79,80,81,82,83,84,85,91,92,93,94,95,96,97,98,99,100,106,107,108,109,110,111,112,113,114,115,121,122,123,124,125,126,127,128,129,130,136,137,138,139,140,141,142,143,144,145,147,149,151,152,153,154,155,156,157,158,159,160) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Anonymized ID            6183
Subject Type             6183
Visit                    6183
Days since enrollment    6183
EID                      6183
START_DATE               6183
Study                    6183
Site                     6183
Year                     6183
Season                   6183
NoDX                     6181
DX_01_Cat                6181
DX_01_Sub                3651
DX_01                    6143
DX_01_Spec                814
DX_01_Code               5964
DX_01_Time               6075
DX_01_Confirmed          3295
DX_01_Presum             3294
DX_01_RC                 6074
DX_01_RuleOut            6075
DX_01_ByHx               5688
DX_01_New                2786
DX_01_Rem                2786
DX_01_PRem               2786
DX_01_Past_Doc            179
DX_02_Cat                3431
DX_02_Sub                1758
DX_02                    3294
DX_02_Spec                844
DX_02_Code               3181
DX_02_Time               5917
DX_02_Confirmed          1952
DX_02_Pres

In [18]:
diag["EID"].nunique()

3616

In [19]:
# Remove duplicated lines
diag_clean = diag.drop_duplicates()
diag_clean.count()

Anonymized ID            3975
Subject Type             3975
Visit                    3975
Days since enrollment    3975
EID                      3975
START_DATE               3975
Study                    3975
Site                     3975
Year                     3975
Season                   3975
NoDX                     3973
DX_01_Cat                3973
DX_01_Sub                2376
DX_01                    3935
DX_01_Spec                580
DX_01_Code               3767
DX_01_Time               3867
DX_01_Confirmed          2229
DX_01_Presum             2228
DX_01_RC                 3866
DX_01_RuleOut            3867
DX_01_ByHx               3649
DX_01_New                1642
DX_01_Rem                1642
DX_01_PRem               1642
DX_01_Past_Doc            112
DX_02_Cat                2347
DX_02_Sub                1121
DX_02                    2209
DX_02_Spec                605
DX_02_Code               2106
DX_02_Time               3709
DX_02_Confirmed          1360
DX_02_Pres

In [20]:
diag_clean["EID"].nunique() 

3616

In [21]:
diag_clean["DX_01"].value_counts() 

ADHD-Combined Type                                                     779
ADHD-Inattentive Type                                                  622
No Diagnosis Given: Incomplete Eval                                    456
No Diagnosis Given                                                     309
Autism Spectrum Disorder                                               295
Specific Learning Disorder with Impairment in Reading                  209
Generalized Anxiety Disorder                                           171
Major Depressive Disorder                                              146
ADHD-Hyperactive/Impulsive Type                                        115
Other Specified Attention-Deficit/Hyperactivity Disorder               106
Social Anxiety (Social Phobia)                                          97
Language Disorder                                                       76
Oppositional Defiant Disorder                                           55
Other Specified Anxiety D

456 incomplete eval

Much more data in COINS than LORIS

In [22]:
# Check if per-assessment LORIS has more diagnosis data than full LORIS
LORIS_diag = pd.read_csv('data/LORIS-diag.csv')
LORIS_diag.count()

Identifiers                    1812
ConsensusDx,Administration     1812
ConsensusDx,Comment_ID            0
ConsensusDx,DX_01              1772
ConsensusDx,DX_01_ByHx         1517
ConsensusDx,DX_01_Cat          1812
ConsensusDx,DX_01_Code         1698
ConsensusDx,DX_01_Confirmed     375
ConsensusDx,DX_01_New          1331
ConsensusDx,DX_01_PRem         1331
ConsensusDx,DX_01_Past_Doc       33
ConsensusDx,DX_01_Presum        375
ConsensusDx,DX_01_RC           1704
ConsensusDx,DX_01_Rem          1331
ConsensusDx,DX_01_RuleOut      1704
ConsensusDx,DX_01_Spec          239
ConsensusDx,DX_01_Sub          1033
ConsensusDx,DX_01_Time         1703
ConsensusDx,DX_02               741
ConsensusDx,DX_02_ByHx          654
ConsensusDx,DX_02_Cat           879
ConsensusDx,DX_02_Code          719
ConsensusDx,DX_02_Confirmed     167
ConsensusDx,DX_02_New           556
ConsensusDx,DX_02_PRem          556
ConsensusDx,DX_02_Past_Doc       26
ConsensusDx,DX_02_Presum        167
ConsensusDx,DX_02_RC        

In [23]:
LORIS_diag["Identifiers"].nunique()

1812

### COINS

In [84]:
# Read relevant coins data (Consensus diagnosis + Questionnnaire Measures of Emotional and Cognitive Status from Assessment_List_Jan2019)
# exclude TRF and TRF_Pre for now, to investigate what to do with multiple entries
assessments = ['ConsensusDx', 'ASR', 'YSR', 'MFQ_SR', 'SCARED_SR', 'C3SR', 'ARI_S', 'CSSRS', 'CIS_SR', 'WHODAS_SR', 
'ICU_SR', 'PANAS', 'CBCL', 'CBCL_Pre', 'MFQ_P', 'SCARED_P', 'ESWAN', 'SWAN', 'ASSQ', 'SCQ', 'CIS_P', 
'SAS', 'WHODAS_P', 'SDQ', 'RBS', 'SRS', 'SRS_Pre', 'ARI_P', 'ICU_P', 'SympChck']

# Read all assessment files into dataframes
coins_file_names = ['9994_' + x + '_20211130.csv' for x in assessments]

all_coins_dfs = []
for file_name in coins_file_names:
    temp_df = pd.read_csv('data/assessment_data/' + file_name, header=1)
    # check if there are assessments with duplicate IDs, because we will aggregate assessments by ID after
    print(file_name, temp_df["ID"].duplicated().any()) # 'TRF', 'TRF_Pre', exclude for now
    all_coins_dfs.append(temp_df)

/var/folders/h4/sy8nk8xs0mb5vmh67_yxsfnw0000gn/T/ipykernel_19374/3184604547.py:12: DtypeWarning: Columns (16,134,147) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv('data/assessment_data/' + file_name, header=1)


9994_ConsensusDx_20211130.csv True
9994_ASR_20211130.csv False
9994_YSR_20211130.csv False
9994_MFQ_SR_20211130.csv False
9994_SCARED_SR_20211130.csv False
9994_C3SR_20211130.csv False
9994_ARI_S_20211130.csv False
9994_CSSRS_20211130.csv False
9994_CIS_SR_20211130.csv False
9994_WHODAS_SR_20211130.csv False
9994_ICU_SR_20211130.csv False
9994_PANAS_20211130.csv False
9994_CBCL_20211130.csv False
9994_CBCL_Pre_20211130.csv False
9994_MFQ_P_20211130.csv False
9994_SCARED_P_20211130.csv False
9994_ESWAN_20211130.csv False
9994_SWAN_20211130.csv False
9994_ASSQ_20211130.csv False
9994_SCQ_20211130.csv False
9994_CIS_P_20211130.csv False
9994_SAS_20211130.csv False
9994_WHODAS_P_20211130.csv False
9994_SDQ_20211130.csv False
9994_RBS_20211130.csv False
9994_SRS_20211130.csv False
9994_SRS_Pre_20211130.csv False
9994_ARI_P_20211130.csv False
9994_ICU_P_20211130.csv False
9994_SympChck_20211130.csv False


In [85]:
# Check that no assessments have duplicated IDs (will aggregate assessment by ID by taking the first non-null value)
for df in all_coins_dfs:
    print(df.columns[4], ": ", df["ID"].count() == df["ID"].nunique())

ConsensusDx_001 :  False
ASRHBN_001 :  True
YSR1_001 :  True
MFQSR_001 :  True
SCAREDSR_001 :  True
C3SR_001 :  True
ARIS1_001 :  True
CSSRS_HBN_001 :  True
CISSR1_001 :  True
WHODASSR_001 :  True
ICUSR_001 :  True
PANAS_HBN_001 :  True
CBCLHBN_001 :  True
CBCL_Pre_001 :  True
MFQP_001 :  True
SCAREDP1_001 :  True
ESWAN_001 :  True
SWAN1_001 :  True
ASSQ1_001 :  True
SCQHBN_001 :  True
CISP_001 :  True
SAS0_001 :  True
WHODASP_001 :  True
SDQHBN_001 :  True
RBS0_001 :  True
SRSHBN_001 :  True
SRSPRE_001 :  True
ARIP1_001 :  True
ICUPHBN_001 :  True
CSC_001 :  True


In [86]:
# Concatenate all assessments
all_coins = pd.concat(all_coins_dfs)
del all_coins_dfs

all_coins.count()

ID                   75067
SUB_TYPE             75067
VISIT                75067
DAY_LAG              75067
ConsensusDx_001       6182
ConsensusDx_002       6182
ConsensusDx_003       6182
ConsensusDx_004       6182
ConsensusDx_005       6182
ConsensusDx_006       6182
ConsensusDx_007       6180
ConsensusDx_008       6180
ConsensusDx_009       3650
ConsensusDx_010       6142
ConsensusDx_011        813
ConsensusDx_012       5963
ConsensusDx_013       6074
ConsensusDx_014       3294
ConsensusDx_015       3293
ConsensusDx_016       6073
ConsensusDx_017       6074
ConsensusDx_018       5687
ConsensusDx_019       2785
ConsensusDx_020       2785
ConsensusDx_021       2785
ConsensusDx_022        178
ConsensusDx_023       3430
ConsensusDx_024       1757
ConsensusDx_025       3293
ConsensusDx_026        843
ConsensusDx_027       3180
ConsensusDx_028       5916
ConsensusDx_029       1951
ConsensusDx_030       1950
ConsensusDx_031       3275
ConsensusDx_032       3275
ConsensusDx_033       3115
C

In [88]:
all_coins["ID"].nunique()

3632

In [89]:
# Group assessements by person
grouped_coins = all_coins.groupby(by=["ID", "SUB_TYPE", "VISIT", "DAY_LAG"]).first()

In [90]:
pd.set_option('display.max_rows', 100) 
grouped_coins = grouped_coins.reset_index()
grouped_coins.count()

ID                 3632
SUB_TYPE           3632
VISIT              3632
DAY_LAG            3632
ConsensusDx_001    3617
                   ... 
CSC_129            3241
CSC_130            3253
CSC_131            3249
CSC_132            3248
CSC_133            3248
Length: 1883, dtype: int64

In [93]:
# Test grouping on a small subset
ASR = pd.read_csv('data/test/' + coins_file_names[1], header=1)
YSR = pd.read_csv('data/test/' + coins_file_names[2], header=1)
MFQ = pd.read_csv('data/test/' + coins_file_names[3], header=1)
ASR_YSR_MFQ_appended = pd.concat([ASR, YSR, MFQ])

In [94]:
grouped_test = ASR_YSR_MFQ_appended.groupby(by=["ID", "SUB_TYPE", "VISIT", "DAY_LAG"])
# grouped_test.apply(display) # output suppressed for privacy

In [128]:
agg_test = grouped_test.first() 
# agg_test # output suppressed for privacy

In [96]:
# Get questionnaires most people answered
pd.set_option('display.max_rows', None)
assessment_column_names = grouped_coins.columns[grouped_coins.columns.str.contains("_001")]
coins_questionnaire_answer_counts = grouped_coins[assessment_column_names].count().sort_values(ascending=False).to_frame()
coins_questionnaire_answer_counts["Ratio"] = coins_questionnaire_answer_counts[0]/total_participants*100
coins_questionnaire_answer_counts

,0,Ratio
ConsensusDx_001,3617,99.587004
ARIS1_001,3278,90.253304
CSC_001,3265,89.895374
SCQHBN_001,3258,89.702643
ASSQ1_001,3235,89.069383
SDQHBN_001,3197,88.023128
ARIP1_001,3190,87.830396
SWAN1_001,3155,86.866740
SRSHBN_001,3105,85.490088
CBCLHBN_001,3098,85.297357


In [102]:
# Remove incomplete Dx

grouped_coins = grouped_coins[grouped_coins["ConsensusDx_010"] != "No Diagnosis Given: Incomplete Eval"]
grouped_coins["ConsensusDx_010"].value_counts()

ADHD-Combined Type                                                     665
ADHD-Inattentive Type                                                  587
No Diagnosis Given                                                     309
Autism Spectrum Disorder                                               272
Specific Learning Disorder with Impairment in Reading                  201
Generalized Anxiety Disorder                                           153
Other Specified Attention-Deficit/Hyperactivity Disorder                99
Social Anxiety (Social Phobia)                                          93
ADHD-Hyperactive/Impulsive Type                                         93
Major Depressive Disorder                                               91
Language Disorder                                                       74
Oppositional Defiant Disorder                                           55
Other Specified Anxiety Disorder                                        48
Intellectual Disability-M

In [100]:
# Re-calculate questionnaires most people answered without incomplete diagnoses

coins_questionnaire_answer_counts = grouped_coins[assessment_column_names].count().sort_values(ascending=False).to_frame()
coins_questionnaire_answer_counts["Ratio"] = coins_questionnaire_answer_counts[0]/total_participants*100
coins_questionnaire_answer_counts

,0,Ratio
ConsensusDx_001,3162,87.059471
SCQHBN_001,3125,86.040749
CSC_001,3122,85.958150
ASSQ1_001,3121,85.930617
ARIP1_001,3085,84.939427
SDQHBN_001,3075,84.664097
SWAN1_001,3044,83.810573
ARIS1_001,3025,83.287445
SRSHBN_001,2967,81.690529
CBCLHBN_001,2953,81.305066


In [107]:
# Get number of people who took all top 1, top 2, top 3, etc. popular assessments

assessments_in_order = coins_questionnaire_answer_counts.index.to_list() # extract assessemnts in order of popularity
cumul_number_of_examples_list = []
for i in range(1, len(assessments_in_order)+1):
    columns = assessments_in_order[0:i] # top i assessments
    cumul_number_of_examples = grouped_coins[columns].notnull().all(axis=1).sum()
    print(columns, ": ", cumul_number_of_examples)
    cumul_number_of_examples_list.append(cumul_number_of_examples)

['ConsensusDx_001'] :  3162
['ConsensusDx_001', 'SCQHBN_001'] :  3116
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001'] :  3107
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001'] :  3096
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001', 'ARIP1_001'] :  3049
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001', 'ARIP1_001', 'SDQHBN_001'] :  3000
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001', 'ARIP1_001', 'SDQHBN_001', 'SWAN1_001'] :  2924
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001', 'ARIP1_001', 'SDQHBN_001', 'SWAN1_001', 'ARIS1_001'] :  2821
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001', 'ARIP1_001', 'SDQHBN_001', 'SWAN1_001', 'ARIS1_001', 'SRSHBN_001'] :  2745
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001', 'ARIP1_001', 'SDQHBN_001', 'SWAN1_001', 'ARIS1_001', 'SRSHBN_001', 'CBCLHBN_001'] :  2744
['ConsensusDx_001', 'SCQHBN_001', 'CSC_001', 'ASSQ1_001', 'ARIP1_001', 'SDQHBN_001', 'SWAN1_001', 'ARIS1_001', 'SRSHBN_001', 'CBCLHBN